# NLP Assessment

The purpose of this project is to analyze a dataset of news articles in two ways:
1. Parsing names and dates from each article in a useable format.
2. Applying text analytics to identify 5 news articles from the dataset that are most similar to a given sample article.

This project is an assessment given by Deloitte for potential Natural Language Processing work with the Centers for Disease Control and Prevention.

# Summary of Results

After analyzing and processing the data, I determined that a Sentence Embedding model was the most accurate at predicting document similarity. A TF-IDF model and a model utilizing Spacy's internal similarity tools were far less performant.

For the final output of this project, I have created two files:
* 'texts_processed.csv': a final csv file of all relevant information, including the columns of names, distinct names, dates and datetime objects.
* 'closest_matches.csv': a sampling of the top five articles in similarity to the sample article, using the Sentence Embedding method.

# Future Recommendations

With more time to devote to this project, there are several changes that could improve accuracy:
* Experimenting with various Spacy models, especially for vectorization/similarity testing.
* Training a custom model/class to detect document similarity utilizing the spacy or flair library.
* Utilizing Spark to increase compute power and decrease run time for text/date classification.

## Ingesting and Analyzing Dataset

The source dataset was provided by Deloitte, as a csv file of unstructured text data from several news articles.

In [2]:
# Importing basic libraries
import pandas as pd
import numpy as np
import datetime

# Importing flair library for name recognition
from flair.data import Sentence, build_spacy_tokenizer
from flair.models import SequenceTagger

# Importing segtok segmenter
from segtok.segmenter import split_single

# Importing spacy for date recognition
import spacy
from spacy.tokenizer import Tokenizer

# Importing ctparse and timefhuman for converting date text into datetime objects
from ctparse import ctparse
from timefhuman import timefhuman

# Importing stopword, tokenizer and stemmer from nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import nltk

# Importing tensorflow hub for Sentence Encoder
import tensorflow as tf
import tensorflow_hub as hub

/Users/eddiekirkland/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.20.4 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/eddiekirkland/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator MultinomialNB from version 0.20.4 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/Users/eddiekirkland/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.4 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [3]:
# Increasing max column width to display text
pd.set_option('max_colwidth', 10000)

In [4]:
# Reading data into dataframe
text_df = pd.read_csv('News-article-wikipedia-DFE.csv')

In [4]:
text_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 14 columns):
 #   Column                                                                                                                                      Non-Null Count  Dtype  
---  ------                                                                                                                                      --------------  -----  
 0   _unit_id                                                                                                                                    3000 non-null   int64  
 1   _unit_state                                                                                                                                 3000 non-null   object 
 2   _trusted_judgments                                                                                                                          3000 non-null   int64  
 3   _last_judgment_at                          

It looks as though there are no null values in the dataset, and 3000 entries for each feature.

In [5]:
# Exploring dataframe
text_df.head()

,_unit_id,_unit_state,_trusted_judgments,_last_judgment_at,given_the_news_article_headline_and_description_above_please_select_the_most_relevant_wikipedia_page_from_the_following_options,given_the_news_article_headline_and_description_above_please_select_the_most_relevant_wikipedia_page_from_the_following_options:confidence,wikipedia_page2__,article,gurl,id,newdescp,nil,option3,oururl
0,691201838,finalized,5,3/19/2015 19:59,Wikipedia Page1 :,1.0000,Wikipedia Page2 :\n\n\nWikipedia Page2 :\nWikipedia Page2 :,Gaza aid ship to dock in Egypt after Israel pressure,http://en.wikipedia.org/wiki/Gaza_Strip,1,"A ship with supplies for Gaza will dock at el-Arish in Egypt, officials say, after Israeli pressure to stop the vessel breaking its Gaza blockade. The Moldovan-flagged ship chartered by a charity run by the son of Libyan leader Col Muammar Gaddafi, left a Greek port on Saturday. Israel asked for help from the UN, and had talks with Greece and Moldova. But organisers insist they will go to Gaza. An Israeli raid on a Gaza-bound ship in May killed nine Turkish activists. Israel insisted its troops were defending themselves but the raid sparked international condemnation. Israel recently eased its blockade, allowing in almost all consumer goods but maintaining a ""blacklist"" of some items. Israel says its blockade of the Palestinian territory is needed to prevent the supply of weapons to the Hamas militant group which controls Gaza. The Amalthea, renamed Hope for the mission, set off from the Greek port of Lavrio, loaded with about 2,000 tonnes of food, cooking oil, medicines and pre-fabricated houses. It has been chartered by the Gaddafi International Charity and Development Foundation. Its chairman is Saif al-Islam Gaddafi. The organisation said the 92m (302ft) vessel would also carry ""a number of supporters who are keen on expressing solidarity with the Palestinian people"".",NaN,None,http://en.wikipedia.org/wiki/Gaza_flotilla_raid
1,691201839,finalized,5,3/19/2015 20:34,Wikipedia Page1 :,1.0000,Wikipedia Page2 :\n\nWikipedia Page2 :\n\n,Mel Gibson,http://en.wikipedia.org/wiki/Mel_Gibson,2,"Often acts and directs stories involving an individual who is persecuted, and fights for justice Has often portrayed a widower, in films such as Mad Max (the sequels), Lethal Weapon film series, Braveheart, The Patriot, Signs, and Edge of Darkness. Often portrays men who seek revenge for the murder of family or friends Ranked #12 in Empire (UK) magazine's ""The Top 100 Movie Stars of All Time"" list. [October 1997] Chosen by People (USA) magazine as one of the ""50 Most Beautiful People"" in the world. Educated at University of New South Wales, Australia. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Awarded the AO (Officer of the Order of Australia), Australia's highest honor, in mid-1997. He took up acting only because his sister submitted an application behind his back. The night before an audition, he got into a fight, and his face was badly beaten, an accident that won him the role.",NaN,None,http://en.wikipedia.org/wiki/Mel_Gibson_filmography
2,691201840,finalized,5,3/19/2015 3:01,Wikipedia Page1 :,1.0000,\n\n\n\n,Talent Agency WME drops Mel Gibson,http://en.wikipedia.org/wiki/Mel_Gibson,3,"Cast member Mel Gibson (R) and Oksana Grigorieva attend the premiere of the film ''Edge of Darkness'' in Los Angeles January 26, 2010. Earlier this week, the agency's Patrick Whitesell informed the actors' representatives that he would no longer be represented by the agency. Gibson's longtime agent, Ed Limato, died July 3, and a funeral will take place in New York next week. William Morris Endeavor (WME) partner Ari Emanuel had previously expressed hostility toward Gibson after the actor made anti-Semitic remarks and made remarks implying skepticism about the Holocaust. An agency source said the only reason the agency had represente

It seems as though much of the information in the dataset is not useful for the purpose of this project. I will simplify the dataframe through selecting relevant columns:
* 'article': the title of each article
* 'oururl': the url of the article's text
* 'newdescp': the basic text of each article

In [5]:
# Keeping only relevant columns
keepcols = ['article', 'oururl', 'newdescp']
text_simple = text_df[keepcols]

# Part 1: Name and Date Recognition

## Name Recognition

To identify names within the text of each article, I will utilize two external libraries:
* Spacy: Used to tokenize the data into sentences. Selected for fast and accurate performance.
* Flair: Used to tokenize spans within each sentence. Selected for its highly intuitive understanding of the context of tokens and token spans.

After identifying a list of names, I will create an additional column containing only distinct names.

**Note:** I considered removing all single-word names such as "Ghadafi," since many are duplicates of the first/last names located within the article. However, simply removing these names could eliminate important information such as people referred to only by last name. For this reason, I left the list as-is.

In [12]:
# Downloading named entity recognition model
tagger = SequenceTagger.load('ner')

# Creating spacy tokenizer for parsing sentences
nlp = spacy.load('en')
tokenizer = build_spacy_tokenizer(nlp)

# Defining function for tagging named entities in text
def find_names(text):
    # Creating empty list for name tokens
    names = []
    sentences = [Sentence(sent, use_tokenizer=tokenizer) for sent in split_single(text)]
    tagger.predict(sentences)
    for sent in sentences:
        for entity in sent.get_spans('ner'):
            if entity.tag=='PER':
                names.append(entity.text)
            else:
                pass
    return names

2020-03-20 15:52:39,619 loading file /Users/eddiekirkland/.flair/models/en-ner-conll03-v0.4.pt


In [13]:
# Applying function to text and creating column of names
text_simple['names'] = text_df['newdescp'].apply(find_names)

2020-03-20 15:59:32,664 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-03-20 15:59:32,666 Ignore 1 sentence(s) with no tokens.
2020-03-20 16:55:47,542 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-03-20 16:55:47,549 Ignore 1 sentence(s) with no tokens.
2020-03-20 17:19:57,990 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-03-20 17:20:30,706 Ignore 1 sentence(s) with no tokens.
2020-03-20 17:41:22,967 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-03-20 17:41:22,969 Ignore 1 sentence(s) with no tokens.
2020-03-20 18:45:08,357 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-03-20 18:45:08,360 Ignore 1 sentence(s) with no tokens.
2020-03-20 19:18:21,727 ACHTUNG: An empty Sentence was created! Are there empty strings in your dataset?
2020-03-20 19:18:21,729 Ignore 1 sentence(s) with no tokens.
2020

/Users/eddiekirkland/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [14]:
# Saving analyzed results to file in case of needed restart
text_simple.to_csv('text_simple_names.csv')

In [15]:
text_simple.head()

,article,oururl,newdescp,names
0,Gaza aid ship to dock in Egypt after Israel pressure,http://en.wikipedia.org/wiki/Gaza_flotilla_raid,"A ship with supplies for Gaza will dock at el-Arish in Egypt, officials say, after Israeli pressure to stop the vessel breaking its Gaza blockade. The Moldovan-flagged ship chartered by a charity run by the son of Libyan leader Col Muammar Gaddafi, left a Greek port on Saturday. Israel asked for help from the UN, and had talks with Greece and Moldova. But organisers insist they will go to Gaza. An Israeli raid on a Gaza-bound ship in May killed nine Turkish activists. Israel insisted its troops were defending themselves but the raid sparked international condemnation. Israel recently eased its blockade, allowing in almost all consumer goods but maintaining a ""blacklist"" of some items. Israel says its blockade of the Palestinian territory is needed to prevent the supply of weapons to the Hamas militant group which controls Gaza. The Amalthea, renamed Hope for the mission, set off from the Greek port of Lavrio, loaded with about 2,000 tonnes of food, cooking oil, medicines and pre-fabricated houses. It has been chartered by the Gaddafi International Charity and Development Foundation. Its chairman is Saif al-Islam Gaddafi. The organisation said the 92m (302ft) vessel would also carry ""a number of supporters who are keen on expressing solidarity with the Palestinian people"".","[Col Muammar Gaddafi, Hope, Saif al, Islam Gaddafi]"
1,Mel Gibson,http://en.wikipedia.org/wiki/Mel_Gibson_filmography,"Often acts and directs stories involving an individual who is persecuted, and fights for justice Has often portrayed a widower, in films such as Mad Max (the sequels), Lethal Weapon film series, Braveheart, The Patriot, Signs, and Edge of Darkness. Often portrays men who seek revenge for the murder of family or friends Ranked #12 in Empire (UK) magazine's ""The Top 100 Movie Stars of All Time"" list. [October 1997] Chosen by People (USA) magazine as one of the ""50 Most Beautiful People"" in the world. Educated at University of New South Wales, Australia. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Awarded the AO (Officer of the Order of Australia), Australia's highest honor, in mid-1997. He took up acting only because his sister submitted an application behind his back. The night before an audition, he got into a fight, and his face was badly beaten, an accident that won him the role.",[]
2,Talent Agency WME drops Mel Gibson,http://en.wikipedia.org/wiki/Lethal_Weapon_(film_series),"Cast member Mel Gibson (R) and Oksana Grigorieva attend the premiere of the film ''Edge of Darkness'' in Los Angeles January 26, 2010. Earlier this week, the agency's Patrick Whitesell informed the actors' representatives that he would no longer be represented by the agency. Gibson's longtime agent, Ed Limato, died July 3, and a funeral will take place in New York next week. William Morris Endeavor (WME) partner Ari Emanuel had previously expressed hostility toward Gibson after the actor made anti-Semitic remarks and made remarks implying skepticism about the Holocaust. An agency source said the only reason the agency had represented Gibson in the first place was his association with Limato. ""Mel was really important to Ed,"" an agency source said. ""He was with him for 32 years and I think Ed saw him as a son."" But he added, ""The world knows how Ari feels and he has never changed that opinion."" Gibson's troubles have only increased in recent weeks with allegations of bigoted tirades and reports that he is under investigation for assaulting his ex-girlfriend. Several studio executives have said in the wake of these disclosures that they consider the troubled actor too untouchable in the industry. ""I'd rather get engaged to Lindsay Lohan than have anything to do with him,"" one studio chief said. A spo

In [68]:
# Removing duplicates from name list
text_simple['names_distinct'] = text_simple['names'].apply(lambda x: list(set(x))).copy()

## Results

Overall, the results of the text recognition seem very reliable. This process took a significant amount of compute power and processing time. In the future, I would experiment with implementing this process on a Spark cluster to increase compute power.

## Date Recognition

Next, I will parse each article to remove any relevant date information. I will do this in two steps:
* Use Spacy's tagging function to identify any date-related words, placing those words in a list within a new column.
* Converting all date-related words in the column to datetime objects, if possible. This should help with future useability of the date information.

In [10]:
# Defining function to detect dates in text
def date_detect(string):
    # Creating tagged doc from string
    doc = nlp(string)
    # Create empty list of dates
    date_list = []
    # Detecting date using spacy libraray
    for ent in doc.ents:
        if ent.label_=='DATE':
            # append date to list
            date_list.append(ent.text)
        else:
            pass
    return date_list

In [11]:
# Applying function to text and creating column of dates
text_simple['dates'] = text_simple['newdescp'].apply(date_detect).copy()

In [12]:
# Define function to apply timefhuman for readable timestamp objects
def timestamp(timelist):
    output = []
    for i in timelist:
        try:
            output.append(timefhuman(i))
        # Exception handling for non readable dates
        except Exception:
            pass
    # Removing empty items
    output = list(filter(None, output))
    return output

In [13]:
# Applying function to text and creating column of datetime objects
text_simple['datetimes'] = text_simple['dates'].apply(timestamp).copy()

In [14]:
text_simple.head()

,Unnamed: 0,article,oururl,newdescp,names,names_distinct,dates,datetimes
0,0,Gaza aid ship to dock in Egypt after Israel pressure,http://en.wikipedia.org/wiki/Gaza_flotilla_raid,"A ship with supplies for Gaza will dock at el-Arish in Egypt, officials say, after Israeli pressure to stop the vessel breaking its Gaza blockade. The Moldovan-flagged ship chartered by a charity run by the son of Libyan leader Col Muammar Gaddafi, left a Greek port on Saturday. Israel asked for help from the UN, and had talks with Greece and Moldova. But organisers insist they will go to Gaza. An Israeli raid on a Gaza-bound ship in May killed nine Turkish activists. Israel insisted its troops were defending themselves but the raid sparked international condemnation. Israel recently eased its blockade, allowing in almost all consumer goods but maintaining a ""blacklist"" of some items. Israel says its blockade of the Palestinian territory is needed to prevent the supply of weapons to the Hamas militant group which controls Gaza. The Amalthea, renamed Hope for the mission, set off from the Greek port of Lavrio, loaded with about 2,000 tonnes of food, cooking oil, medicines and pre-fabricated houses. It has been chartered by the Gaddafi International Charity and Development Foundation. Its chairman is Saif al-Islam Gaddafi. The organisation said the 92m (302ft) vessel would also carry ""a number of supporters who are keen on expressing solidarity with the Palestinian people"".","['Col Muammar Gaddafi', 'Hope', 'Saif al', 'Islam Gaddafi']","[a, I, ], S, l, M, C, m, f, i, o, u, r, d, s, p, ,, , [, H, G, ', e]","[Saturday, May]",[2020-03-28 00:00:00]
1,1,Mel Gibson,http://en.wikipedia.org/wiki/Mel_Gibson_filmography,"Often acts and directs stories involving an individual who is persecuted, and fights for justice Has often portrayed a widower, in films such as Mad Max (the sequels), Lethal Weapon film series, Braveheart, The Patriot, Signs, and Edge of Darkness. Often portrays men who seek revenge for the murder of family or friends Ranked #12 in Empire (UK) magazine's ""The Top 100 Movie Stars of All Time"" list. [October 1997] Chosen by People (USA) magazine as one of the ""50 Most Beautiful People"" in the world. Educated at University of New South Wales, Australia. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Awarded the AO (Officer of the Order of Australia), Australia's highest honor, in mid-1997. He took up acting only because his sister submitted an application behind his back. The night before an audition, he got into a fight, and his face was badly beaten, an accident that won him the role.",[],"[], []","[October 1997, mid-1997]",[1997-10-01 00:00:00]
2,2,Talent Agency WME drops Mel Gibson,http://en.wikipedia.org/wiki/Lethal_Weapon_(film_series),"Cast member Mel Gibson (R) and Oksana Grigorieva attend the premiere of the film ''Edge of Darkness'' in Los Angeles January 26, 2010. Earlier this week, the agency's Patrick Whitesell informed the actors' representatives that he would no longer be represented by the agency. Gibson's longtime agent, Ed Limato, died July 3, and a funeral will take place in New York next week. William Morris Endeavor (WME) partner Ari Emanuel had previously expressed hostility toward Gibson after the actor made anti-Semitic remarks and made remarks implying skepticism about the Holocaust. An agency source said the only reason the agency had represented Gibson in the first place was his association with Limato. ""Mel was really important to Ed,"" an agency source said. ""He was with him for 32 years and I think Ed saw him as a son."" But he added, ""The world knows how Ari feels and he has never changed that opinion."" Gibson's troubles have only increased in recent weeks with allegations of bigoted tirades and reports that he is under investigation for assaulting his ex-girlfriend. Several studio executi

## Results

After the data processing, we have several new columns of relevant data:
* 'names': a list containing all personal names detected in the article
* 'names_distinct': a list of all distinct names detected
* 'dates': a list of all date-type strings detected
* 'datetimes': a list of datetime objects for any detectable timestamps

**Note:** The datetime objects could be more accurate if each article had a corresponding date of publication. This could allow the library to index those dates such as "last week" or "next Tuesday" to specific timestamps. From the given data, we might have been able to use the `'last_judgement_at'` column, but without documentation we cannot tell if this is an appopriate reference date to use.

# Part 2: Check for Document Similarity

For part two of this assessment, we will scan the news articles to find those most similar to a provided sample. To do this, we will try a few machine learning techniques:
* "Term Frequency - Inverse Document Freqency" (TF-IDF) with a cosine similarity test
* Sentence Emedding with Google Sentence Encoder
* Spacy Vectorization - using Spacy's internal vectorization and similariy function

## Preprocessing
To begin, we need to preprocess the data in order to:
* Lowercase all words
* Remove stop words
* Remove puntuation
* Remove single characters (unneeeded for our analysis)
* Stem each word
* Lemmatize each word
* Convert numbers to words

### Creating Preprocessing Functions

In [143]:
# Function to lowercase all words
def convert_lower_case(text):
    return np.char.lower(text)

In [144]:
# Function to remove stop words
def remove_stop_words(text):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(text))
    new_text = ""
    for w in words:
        if w not in stop_words and len(w) > 1:
            new_text = new_text + " " + w
    return new_text

In [145]:
# Function to remove punctuation
def remove_punctuation(text):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        text = np.char.replace(text, symbols[i], ' ')
        text = np.char.replace(text, " ", " ")
    text = np.char.replace(text, ',', '')
    return text

In [146]:
# Function to remove apostrophes
# We call this function separately to avoid erros in possessive words
def remove_apostrophe(text):
    return np.char.replace(text, "'", '')

In [147]:
# Function to stem words
def stemming(text):
    stemmer = PorterStemmer()
    tokens = word_tokenize(str(text))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return new_text

In [153]:
# Function to convert numbers to text
def convert_numbers(text):
    tokens = word_tokenize(str(text))
    new_text = ""
    for w in tokens:
        try:
            w = num2words(int(w))
        except:
            a = 0
        new_text = new_text + " " + w
    new_text = np.char.replace(new_text, "-", " ")
    return new_text

In [158]:
# Function to preprocess data using previously defined functions
def preprocess(text):
    text = convert_lower_case(text)
    text = remove_punctuation(text)
    text = remove_apostrophe(text)
    text = remove_stop_words(text)
    text = convert_numbers(text)
    text = stemming(text)
    text = remove_punctuation(text)
    text = convert_numbers(text)
    text = stemming(text)
    text = remove_punctuation(text) # needed again as num2word adds some hyphens and commas
    text = remove_stop_words(text) # needed again as num2word adds some stop words
    return text

In [161]:
# Applying preprocessing function to all articles
text_simple['processed_text'] = text_simple['newdescp'].apply(preprocess).copy()

In [162]:
text_simple.head()

,article,oururl,newdescp,names,dates,datetimes,names_distinct,processed_text
0,Gaza aid ship to dock in Egypt after Israel pressure,http://en.wikipedia.org/wiki/Gaza_flotilla_raid,"A ship with supplies for Gaza will dock at el-Arish in Egypt, officials say, after Israeli pressure to stop the vessel breaking its Gaza blockade. The Moldovan-flagged ship chartered by a charity run by the son of Libyan leader Col Muammar Gaddafi, left a Greek port on Saturday. Israel asked for help from the UN, and had talks with Greece and Moldova. But organisers insist they will go to Gaza. An Israeli raid on a Gaza-bound ship in May killed nine Turkish activists. Israel insisted its troops were defending themselves but the raid sparked international condemnation. Israel recently eased its blockade, allowing in almost all consumer goods but maintaining a ""blacklist"" of some items. Israel says its blockade of the Palestinian territory is needed to prevent the supply of weapons to the Hamas militant group which controls Gaza. The Amalthea, renamed Hope for the mission, set off from the Greek port of Lavrio, loaded with about 2,000 tonnes of food, cooking oil, medicines and pre-fabricated houses. It has been chartered by the Gaddafi International Charity and Development Foundation. Its chairman is Saif al-Islam Gaddafi. The organisation said the 92m (302ft) vessel would also carry ""a number of supporters who are keen on expressing solidarity with the Palestinian people"".","[Col Muammar Gaddafi, Hope, Saif al, Islam Gaddafi]","[Saturday, May]",[2020-03-21 00:00:00],"[Col Muammar Gaddafi, Hope, Islam Gaddafi, Saif al]",ship suppli gaza dock el arish egypt offici say isra pressur stop vessel break gaza blockad moldovan flag ship charter chariti run son libyan leader col muammar gaddafi left greek port saturday israel ask help un talk greec moldova organi insist go gaza isra raid gaza bound ship may kill nine turkish activist israel insist troop defend raid spark intern condemn israel recent ea blockad allow almost consum good maintain blacklist item israel say blockad palestinian territori need prevent suppli weapon hama milit group control gaza amalthea renam hope mission set greek port lavrio load two thousand tonn food cook oil medicin pre fabric hou charter gaddafi intern chariti develop foundat chairman saif al islam gaddafi organi said 92m 302ft vessel would also carri number support keen express solidar palestinian peopl
1,Mel Gibson,http://en.wikipedia.org/wiki/Mel_Gibson_filmography,"Often acts and directs stories involving an individual who is persecuted, and fights for justice Has often portrayed a widower, in films such as Mad Max (the sequels), Lethal Weapon film series, Braveheart, The Patriot, Signs, and Edge of Darkness. Often portrays men who seek revenge for the murder of family or friends Ranked #12 in Empire (UK) magazine's ""The Top 100 Movie Stars of All Time"" list. [October 1997] Chosen by People (USA) magazine as one of the ""50 Most Beautiful People"" in the world. Educated at University of New South Wales, Australia. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Chosen by People magazine as one of the ""50 Most Beautiful People"" in the world. Awarded the AO (Officer of the Order of Australia), Australia's highest honor, in mid-1997. He took up acting only because his sister submitted an application behind his back. The night before an audition, he got into a fight, and his face was badly beaten, an accident that won him the role.",[],"[October 1997, mid-1997]",[1997-10-01 00:00:00],[],often act direct stori involv individu persecut fight justic often portray widow film mad max sequel lethal weapon film seri braveheart patriot sign edg dark often portray men seek reveng murder famili friend rank twelv empir uk magazin top one hundr movi star time list octob one thousand nine hundr nineti seven chosen peopl usa magazin one fifti beauti peopl world educ univ new south wale australia cho

### Treating Sample Text

In [164]:
# Importing sample text
sample = '''
         Human Rights Watch says government-controlled health services in Egypt have been pressured into playing down the number of casualties during anti-government protests. The group has documented the deaths of 297 people, but says the final toll is likely to be significantly higher. Human Rights Watch says the vast majority of the deaths in Cairo, Alexandria and Suez were on January 28 and 29 as a result of live gunfire as riot police fought running battles with protesters. A significant proportion came as a result of rubber bullets fired at too close a range and from teargas canisters fired into the crowds at very close range. Human Rights Watch says the actual number of deaths is likely to be an underestimate because the organisation had only included those deaths it had verified itself at key hospitals in the three major cities.
         '''
# Preprocessing sample to match dataset
sample = preprocess(sample)

In [165]:
sample

' human right watch say govern control health servic egypt pressur play number casualti anti govern protest group document death two hundr nineti seven peopl say final toll like significantli higher human right watch say vast major death cairo alexandria suez januari twenti eight twenti nine result live gunfir riot polic fought run battl protest signif proport came result rubber bullet fire close rang tearga canist fire crowd close rang human right watch say actual number death like underestim organi includ death verifi key hospit three major citi'

## Test 1: TF-IDF Cosine Similarity

In [166]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [167]:
# defining vectorization function for texts
def get_vectors(strings):
    text = [t for t in strings]
    vectorizer = CountVectorizer(text)
    vectorizer.fit(text)
    return vectorizer.transform(text).toarray()

In [178]:
# defining cosine similarity function
def get_cosine_sim(strings):
    vectors = [t for t in get_vectors(strings)]
    return cosine_similarity(vectors)[1,0]

In [182]:
# defining function to combine sample with each item
def cosine_test(item):
    sample = '''
         Human Rights Watch says government-controlled health services in Egypt have been pressured into playing down the number of casualties during anti-government protests. The group has documented the deaths of 297 people, but says the final toll is likely to be significantly higher. Human Rights Watch says the vast majority of the deaths in Cairo, Alexandria and Suez were on January 28 and 29 as a result of live gunfire as riot police fought running battles with protesters. A significant proportion came as a result of rubber bullets fired at too close a range and from teargas canisters fired into the crowds at very close range. Human Rights Watch says the actual number of deaths is likely to be an underestimate because the organisation had only included those deaths it had verified itself at key hospitals in the three major cities.
         '''
    test_items = [item, sample]
    return get_cosine_sim(test_items)

In [194]:
# Applying cosine test to all articles
text_simple['cosine_test'] = text_simple['processed_text'].apply(cosine_test).copy()

In [195]:
text_simple = text_simple.sort_values(by='cosine_test', ascending=False)

In [196]:
text_simple.head()

,article,oururl,newdescp,names,dates,datetimes,names_distinct,processed_text,cosine_test
76,Egypt hospitals 'told to downplay protest deaths',http://en.wikipedia.org/wiki/Egyptian_Revolution_of_2011,"Human Rights Watch says government-controlled health services in Egypt have been pressured into playing down the number of casualties during anti-government protests. The group has documented the deaths of 297 people, but says the final toll is likely to be significantly higher. Human Rights Watch says the vast majority of the deaths in Cairo, Alexandria and Suez were on January 28 and 29 as a result of live gunfire as riot police fought running battles with protesters. A significant proportion came as a result of rubber bullets fired at too close a range and from teargas canisters fired into the crowds at very close range. Human Rights Watch says the actual number of deaths is likely to be an underestimate because the organisation had only included those deaths it had verified itself at key hospitals in the three major cities.",[],[January 28 and 29],[],[],human right watch say govern control health servic egypt pressur play number casualti anti govern protest group document death two hundr nineti seven peopl say final toll like significantli higher human right watch say vast major death cairo alexandria suez januari twenti eight twenti nine result live gunfir riot polic fought run battl protest signif proport came result rubber bullet fire close rang tearga canist fire crowd close rang human right watch say actual number death like underestim organi includ death verifi key hospit three major citi,0.223969
2361,Libya: Security Forces Kill 84 Over Three Days,http://en.wikipedia.org/wiki/Libyan_Civil_War_(2011),"Muammar Gaddafi's security forces are firing on Libyan citizens and killing scores simply because they're demanding change and accountability. Libyan authorities should allow peaceful protesters to have their say. (New York) - Government security forces have killed at least 84 people in three days of protests in several cities in Libya, Human Rights Watch said today, based on telephone interviews with local hospital staff and witnesses. The Libyan authorities should immediately end attacks on peaceful protesters and protect them from assault by pro-government armed groups, Human Rights Watch said. Thousands of demonstrators gathered in the eastern Libyan cities of Benghazi, Baida, Ajdabiya, Zawiya, and Derna on February 18, 2011, following violent attacks against peaceful protests the day before that killed 20 people in Benghazi, 23 in Baida, three in Ajdabiya, and three in Derna. Hospital sources told Human Rights Watch that security forces killed 35 people in Benghazi on February 18, almost all with live ammunition. ""Muammar Gaddafi's security forces are firing on Libyan citizens and killing scores simply because they're demanding change and accountability,"" said Joe Stork, deputy Middle East and North Africa director at Human Rights Watch. ""Libyan authorities should allow peaceful protesters to have their say."" The protests in Benghazi on February 18 began during funerals for the 20 demonstrators killed by security forces the day before. Eyewitnesses told Human Rights Watch that security forces with distinctive yellow uniforms opened fire on protesters near the Fadil Bu Omar Katiba, a security force base in the center of Benghazi. One protester told Human Rights Watch he witnessed four men shot dead. By 11 p.m. on February 18, Al Jalaa Hospital in Benghazi had received the bodies of 35 people killed that day, a senior hospital official told Human Rights Watch. He said the deaths had been caused by gunshot wounds to the chest, neck, and head. Two sources at the hospital confirmed to Human Rights Watch that the death toll for February 17 was 20, and that at least 45 people had been wounded by bullets. The senior hospital official told Human Rights Watch, ""We put out a call to all the doctors in Benghazi to come to the

From this test, we can tell that the sample text is still represented in the dataset. For the purposes of our testing, we will leave the sample text in the dataset to ensure that our similarity testing is working appropriately. In each case, this should be our top result. For now, let's explore the remaining top 5 articles in similarity.

In [205]:
cosine_samples = text_simple.sort_values(by='cosine_test', ascending=False).head(6)
cosine_samples = cosine_samples[1:]

In [206]:
cosine_samples.head()

,article,oururl,newdescp,names,dates,datetimes,names_distinct,processed_text,cosine_test
2361,Libya: Security Forces Kill 84 Over Three Days,http://en.wikipedia.org/wiki/Libyan_Civil_War_(2011),"Muammar Gaddafi's security forces are firing on Libyan citizens and killing scores simply because they're demanding change and accountability. Libyan authorities should allow peaceful protesters to have their say. (New York) - Government security forces have killed at least 84 people in three days of protests in several cities in Libya, Human Rights Watch said today, based on telephone interviews with local hospital staff and witnesses. The Libyan authorities should immediately end attacks on peaceful protesters and protect them from assault by pro-government armed groups, Human Rights Watch said. Thousands of demonstrators gathered in the eastern Libyan cities of Benghazi, Baida, Ajdabiya, Zawiya, and Derna on February 18, 2011, following violent attacks against peaceful protests the day before that killed 20 people in Benghazi, 23 in Baida, three in Ajdabiya, and three in Derna. Hospital sources told Human Rights Watch that security forces killed 35 people in Benghazi on February 18, almost all with live ammunition. ""Muammar Gaddafi's security forces are firing on Libyan citizens and killing scores simply because they're demanding change and accountability,"" said Joe Stork, deputy Middle East and North Africa director at Human Rights Watch. ""Libyan authorities should allow peaceful protesters to have their say."" The protests in Benghazi on February 18 began during funerals for the 20 demonstrators killed by security forces the day before. Eyewitnesses told Human Rights Watch that security forces with distinctive yellow uniforms opened fire on protesters near the Fadil Bu Omar Katiba, a security force base in the center of Benghazi. One protester told Human Rights Watch he witnessed four men shot dead. By 11 p.m. on February 18, Al Jalaa Hospital in Benghazi had received the bodies of 35 people killed that day, a senior hospital official told Human Rights Watch. He said the deaths had been caused by gunshot wounds to the chest, neck, and head. Two sources at the hospital confirmed to Human Rights Watch that the death toll for February 17 was 20, and that at least 45 people had been wounded by bullets. The senior hospital official told Human Rights Watch, ""We put out a call to all the doctors in Benghazi to come to the hospital and for everyone to contribute blood because I've never seen anything like this before."" Witnesses said that after the February 18 shootings, protesters in Benghazi continued on to the courthouse and gathered there throughout the evening, the crowd swelling to thousands. In Baida, further to the east, protesters on February 18 buried the 23 people who had been shot dead the day before. One protester told Human Rights Watch that police were patrolling the streets but he had seen no further clashes.","[Muammar Gaddafi, Muammar Gaddafi, Joe Stork]","[three days, today, February 18, 2011, the day, February 18, February 18, the day before, February 18, that day, February 17, 20, February 18, February 18, the day before]","[2020-03-21 00:00:00, 2011-02-18 00:00:00, 2020-02-18 00:00:00, 2020-02-18 00:00:00, 2020-02-18 00:00:00, 2020-02-17 00:00:00, 2020-03-21 20:00:00, 2020-02-18 00:00:00, 2020-02-18 00:00:00]","[Muammar Gaddafi, Joe Stork]",muammar gaddafi secur forc fire libyan citizen kill score simpli theyr demand chang account libyan author allow peac protest say new york govern secur forc kill least eighti four peopl three day protest sever citi libya human right watch said today base telephon interview local hospit staff wit libyan author immedi end attack peac protest protect assault pro govern arm group human right watch said thousand demonstr gather eastern libyan citi benghazi baida ajdabiya zawiya derna februari eighteen two thousand eleven follow violent attack peac protest day kill twenti peopl beng

### Test 1: Results

From exploring the top 5 articles by cosine similarity, it seems that articles 1, 2 and 5 have a good deal of contextual overlap. These artciles refer to protests and human rights abuses in Libya and Syria, while the sample text is about human rights abuses from protests in Egypt.

However, articles 3 and 4 are related mainly to viruses and have nothing to do protests, human rights, or the middle east. Further similarity testing is needed to refine results.

## Test 2: Sentence Embedding

For this test, we will utilize the Google Sentence Encoder. This encoder uses embeddings for words within larger sentence structures which are averaged together and given an embedding for the overall sentence. These larger embeddings are compared for correlation.

Because this model utilizes sentence structure, we will use the raw text from each article rather than its cleaned and tokenized version.

In [10]:
# Import module from Universal Sentence Encoder
module_url = "https://tfhub.dev/google/universal-sentence-encoder/4" #@param ["https://tfhub.dev/google/universal-sentence-encoder/4", "https://tfhub.dev/google/universal-sentence-encoder-large/5"]
model = hub.load(module_url)
print ("module %s loaded" % module_url)

# Define function to embed sentences.
def embed(input):
  return model(input)

INFO:absl:Using /var/folders/1_/yqjpttyx6yvg73s_v4rn7_gc0000gn/T/tfhub_modules to cache modules.
INFO:absl:Downloading TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 180.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 400.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 620.00MB
INFO:absl:Downloading https://tfhub.dev/google/universal-sentence-encoder/4: 850.00MB
INFO:absl:Downloaded https://tfhub.dev/google/universal-sentence-encoder/4, Total size: 987.47MB
INFO:absl:Downloaded TF-Hub Module 'https://tfhub.dev/google/universal-sentence-encoder/4'.


module https://tfhub.dev/google/universal-sentence-encoder/4 loaded


In [11]:
# Define a function to score similarity based on sentence enbeddings
def embedding_corr(test_items):
  features = embed(test_items)
  # Return correlation between sentence embeddings
  # Using absolute value to standardize correlations
  corr = np.abs(np.inner(features, features))
  return corr[1,0]

# defining function to combine sample with each item
def embedding_test(item):
    sample = '''
         Human Rights Watch says government-controlled health services in Egypt have been pressured into playing down the number of casualties during anti-government protests. The group has documented the deaths of 297 people, but says the final toll is likely to be significantly higher. Human Rights Watch says the vast majority of the deaths in Cairo, Alexandria and Suez were on January 28 and 29 as a result of live gunfire as riot police fought running battles with protesters. A significant proportion came as a result of rubber bullets fired at too close a range and from teargas canisters fired into the crowds at very close range. Human Rights Watch says the actual number of deaths is likely to be an underestimate because the organisation had only included those deaths it had verified itself at key hospitals in the three major cities.
         '''
    test_items = [item, sample]
    return embedding_corr(test_items)

In [12]:
# Applying sentence embedding test to all articles
text_simple['sentence_test'] = text_simple['newdescp'].apply(embedding_test).copy()

In [13]:
sentence_samples = text_simple.sort_values(by='sentence_test', ascending=False).head(6)
sentence_samples = sentence_samples[1:]
sentence_samples.head()

,Unnamed: 0,article,oururl,newdescp,names,dates,datetimes,names_distinct,processed_text,cosine_test,sentence_test,spacy_test
89,585,Deaths in Egypt's Suez after Port Said football unrest,http://en.wikipedia.org/wiki/2012?13_Egyptian_protests,"Two people have been killed and more than 400 injured in protests across Egypt sparked by the deaths of 74 people after a football match. The two killed were shot by police trying to disperse angry crowds in the city of Suez, medical officials said. In the capital Cairo, thousands of protesters remained on the streets following a day of clashes with police. Thousands marched to the interior ministry, where security forces fired tear gas to keep them back. Earlier, the Egyptian prime minister announced the sackings of several senior officials. Funerals of some of the 74 victims took place in Port Said, where the football match had taken place on Wednesday. The deaths came when fans invaded the pitch after a fixture between top Cairo club al-Ahly and the Port Said side al-Masry. As night fell in Cairo, several thousand demonstrators remained in the streets around the interior ministry, witnesses said. In Suez, health official Mohammed Lasheen said two people had been shot dead early on Friday. A witness quoted by Reuters said: ""Protesters are trying to break into the Suez police station and police are now firing live ammunition.""","['Masry', 'Mohammed Lasheen']","['a day', 'Wednesday', 'Friday']","[datetime.datetime(2020, 3, 25, 0, 0), datetime.datetime(2020, 3, 27, 0, 0)]","['Masry', 'Mohammed Lasheen']",two peopl kill four hundr injur protest across egypt spark death seventi four peopl footbal match two kill shot polic tri disper angri crowd citi suez medic offici said capit cairo thousand protest remain street follow day clash polic thousand march interior ministri secur forc fire tear ga keep back earlier egyptian prime minist announc sack sever senior offici funer seventi four victim took place port said footbal match taken place wednesday death came fan invad pitch fixtur top cairo club al ahli port said side al masri night fell cairo sever thousand demonstr remain street around interior ministri wit said suez health offici moham lasheen said two peopl shot dead earli friday wit quot reuter said protest tri break suez polic station polic fire live ammunit,0.033836,0.708794,0.844297
1,2361,Libya: Security Forces Kill 84 Over Three Days,http://en.wikipedia.org/wiki/Libyan_Civil_War_(2011),"Muammar Gaddafi's security forces are firing on Libyan citizens and killing scores simply because they're demanding change and accountability. Libyan authorities should allow peaceful protesters to have their say. (New York) - Government security forces have killed at least 84 people in three days of protests in several cities in Libya, Human Rights Watch said today, based on telephone interviews with local hospital staff and witnesses. The Libyan authorities should immediately end attacks on peaceful protesters and protect them from assault by pro-government armed groups, Human Rights Watch said. Thousands of demonstrators gathered in the eastern Libyan cities of Benghazi, Baida, Ajdabiya, Zawiya, and Derna on February 18, 2011, following violent attacks against peaceful protests the day before that killed 20 people in Benghazi, 23 in Baida, three in Ajdabiya, and three in Derna. Hospital sources told Human Rights Watch that security forces killed 35 people in Benghazi on February 18, almost all with live ammunition. ""Muammar Gaddafi's security forces are firing on Libyan citizens and killing scores simply because they're demanding change and accountability,"" said Joe Stork, deputy Middle East and North Africa director at Human Rights Watch. ""Libyan authorities should allow peaceful protesters to have their say."" The protests in Benghazi on February 18 began during funerals for the 20 demonstrators killed by security forces the day before. Eyewitnesses told Human Rights Watch that security forc

### Test 2: Results

The results of the sentence encoder seem much more promising. Overall, all five articles share many similarities. They all refer to violent protests, and all are located somewhere in the Middle East/North Africa region (Egypt, Libya, Yemen). Several refer to human rights issues and anti-government protests, much like the sample article.

The results of this test seem far more accurate than the cosine similarity test.

## Test 3: Spacy Vectorization

Finally, we will run a similarity test using Spacy's internal similarity function. This model vectorizes the sample and article texts using spacy's nlp vectorizer utilized above. It then compares similarity in both directions (sample->test, test->sample) and averages the similarity score.

For this test, we will utilize the tokenized and cleaned version of the article text.

In [5]:
# Loading spacy vectorized model - en_core_web_md
nlp = spacy.load("en_core_web_md")

In [6]:
# Defining function to use spacy's vectorizer
def spacy_test(string):
    # Checking to make sure each value contains information
    if string:
        # Storing sample article and string, vectorizing using spacy nlp
        sample = nlp('''
             Human Rights Watch says government-controlled health services in Egypt have been pressured into playing down the number of casualties during anti-government protests. The group has documented the deaths of 297 people, but says the final toll is likely to be significantly higher. Human Rights Watch says the vast majority of the deaths in Cairo, Alexandria and Suez were on January 28 and 29 as a result of live gunfire as riot police fought running battles with protesters. A significant proportion came as a result of rubber bullets fired at too close a range and from teargas canisters fired into the crowds at very close range. Human Rights Watch says the actual number of deaths is likely to be an underestimate because the organisation had only included those deaths it had verified itself at key hospitals in the three major cities.
             ''')
        testdoc = nlp(string)
        # Testing for similarity in both directions
        similarity = sample.similarity(testdoc)
        similarity_rev = testdoc.similarity(sample)
        # Averaging both similarity scores
        sim_score = (similarity + similarity_rev) / 2
    else:
        pass
    return sim_score

In [7]:
# Applying spacy vectorization test to all articles
text_simple['spacy_test'] = text_simple['processed_text'].apply(spacy_test).copy()

In [8]:
spacy_samples = text_simple.sort_values(by='spacy_test', ascending=False).head(6)
spacy_samples = spacy_samples[1:]
spacy_samples.head()

,Unnamed: 0,article,oururl,newdescp,names,dates,datetimes,names_distinct,processed_text,cosine_test,sentence_test,spacy_test
35,1188,"UPDATE 1-Libya said to use cluster arms, Tripoli denies it",http://en.wikipedia.org/wiki/Timeline_of_the_2011_Libyan_Civil_War,"Cluster munitions -- fired from artillery or rockets -- can scatter bomblets over a wide area. They often fail to detonate immediately and may explode years after a conflict, killing or maiming people, according to humanitarian groups. Human Rights Watch said in a statement that government forces had fired cluster munitions into residential areas in the western city of Misrata, ""posing a grave risk to civilians"". The group said it had observed at least three such weapons explode over the coastal city's el-Shawahda neighbourhood on the night of April 14. ""Researchers inspected the remnants of a cluster submunition and interviewed witnesses to two other apparent cluster munition strikes,"" it said on its website, which also had photographs of what it said were the remnants of the munitions used in Misrata. ""They pose a huge risk to civilians, both during attacks because of their indiscriminate nature and afterward because of the still-dangerous unexploded duds scattered about,"" he said. Based on a submunition first discovered by a New York Times reporter and inspected by Human Rights Watch, the group said the cluster munition was a Spanish-produced MAT-120 120 mm mortar projectile, which it said opens in mid-air and releases 21 submunitions over a wide area. It said it had not yet been able to determine if civilians in Misrata had been wounded or killed by such munitions. A rebel spokesman in Misrata, the insurgents' last major enclave in western Libya, also told Reuters cluster munitions had been used by pro-government forces there. ""To use these bombs, the evidence would remain for days and weeks, and we know the international community is coming en masse to our country soon, so we can't do this."" Libya has invited UNICEF to visit Misrata and on Saturday a Red Crescent and Red Cross team will go there, he said.",[],"['may explode years', 'days', 'weeks', 'Saturday']","[datetime.datetime(2020, 5, 1, 0, 0), datetime.datetime(2020, 3, 21, 0, 0)]",[],cluster munit fire artilleri rocket scatter bomblet wide area often fail deton immedi may explod year conflict kill maim peopl accord humanitarian group human right watch said statement govern forc fire cluster munit residenti area western citi misrata pose grave risk civilian group said observ least three weapon explod coastal citi el shawahda neighbourhood night april fourteen research inspect remnant cluster submunit interview wit two appar cluster munit strike said websit also photograph said remnant munit use misrata pose huge risk civilian attack indiscrimin natur afterward still danger unexplod dud scatter said base submunit first discov new york time report inspect human right watch group said cluster munit spanish produc mat one hundr twenti one hundr twenti mm mortar projectil said open mid air relea twenti one submunit wide area said yet abl determin civilian misrata wound kill munit rebel spokesman misrata insurg last major enclav western libya also told reuter cluster munit use pro govern forc use bomb evid would remain day week know intern commun come en mass countri soon cant libya invit unicef visit misrata saturday red crescent red cross team go said,0.043845,0.563499,0.914272
53,8,US admits Afghan airstrike errors,http://en.wikipedia.org/wiki/Drone_strikes_in_Pakistan,"A row has been rumbling since the strikes in early May Failure by US forces to follow their own rules was the ""likely"" cause of civilian deaths in Afghan airstrikes last month, a US military report says. US officials looked at seven strikes on Taliban targets in Farah province on 4 May, and concluded that three had not complied with military guidelines. The report accepts that at least 26 civilians died, but acknowledges that the real f

### Test 3 Results

This test required a significant amount more compute power than the others, and its results are similar to those of Test 1. Two articles seem to be related to civil unrest in the middle east region. The others, however, relate to a child sex abuse scandal in Cork, an estimate of civilian death toll in Iraq, and an article on Typhoon Haian in the Philippines.

On the whole, this model seems to be the least accurate.

# Part 2: Results

After processing the text using three separate tools, it seems that the Sentence Encoding test provides the most accurate result.

# Outputting Final Results to Files

Finally, I will output two files:
* 'texts_processed.csv': a final csv file of all relevant information, including the columns of names, distinct names, dates and datetime objects.
* 'closest_matches.csv': a sampling of the top five articles in similarity to the sample article, using the Sentence Embedding method.

In [14]:
text_simple.to_csv('texts_processed.csv')
sentence_samples.to_csv('closest_matches.csv')